In [ ]:
import torch
import whisper
from transformers import AutoModelForCausalLM, AutoTokenizer
import intel_extension_for_pytorch as ipex

# Func to free up XPU VRAM from allocator
def clearvram():
    torch.xpu.memory.empty_cache()

# Clear VRAM
clearvram()

# Load a pre-trained model (e.g., "base", "small", "medium", "large")
model = whisper.load_model("turbo", device="xpu") 

# Transcribe your audio file
result = model.transcribe("test.wav")

print(result["text"])

model_id = "microsoft/Phi-3-mini-4k-instruct" # Replace with your chosen model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="xpu", torch_dtype=torch.bfloat16) # Adjust dtype as needed

prompt = result["text"]

inputs = tokenizer(prompt, return_tensors="pt", padding=True).to("xpu")

# Generate text
output_ids = model.generate(inputs.input_ids, max_new_tokens=150, num_return_sequences=1)

# Decode the generated text
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_text)

# Clear VRAM
clearvram()

In [ ]:
import torch
import whisper
from transformers import AutoModelForCausalLM, AutoTokenizer

# Func to free up XPU VRAM from allocator
def clearvram():
    torch.xpu.memory.empty_cache()

device = "cpu"

# Clear VRAM
clearvram()

# Load a pre-trained model (e.g., "base", "small", "medium", "large")
model = whisper.load_model("turbo", device="cpu") 

# Transcribe your audio file
result = model.transcribe("test.wav")

print(result["text"])

model_id = "microsoft/Phi-3-mini-4k-instruct" # Replace with your chosen model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16) # Adjust dtype as needed

prompt = result["text"]

inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(device)

# Generate text
output_ids = model.generate(inputs.input_ids, max_new_tokens=150, num_return_sequences=1)

# Decode the generated text
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_text)

In [1]:
import torch
import whisper
from transformers import AutoModelForCausalLM, AutoTokenizer

# Func to free up XPU VRAM from allocator
def clearvram():
    torch.xpu.memory.empty_cache()

# Clear VRAM
clearvram()

# Load a pre-trained model (e.g., "base", "small", "medium", "large")
whisper_model = whisper.load_model("turbo", device="xpu")
result = whisper_model.transcribe("test2.wav")
transcript = result["text"]
print("Transcript:", transcript)

model_id = "microsoft/Phi-3-mini-4k-instruct" # Replace with your chosen model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="xpu", torch_dtype=torch.bfloat16) # Adjust dtype as needed

model.eval()

# === 3. Build PROPER chat prompt for Phi-3 ===
messages = [
    {"role": "system", "content": "You are a helpful assistant that summarizes podcast transcripts concisely."},
    {"role": "user", "content": f"Please summarize the following podcast excerpt in 2-3 sentences:\n\n{transcript}"}
]

# Apply chat template
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True  # This adds <|assistant|>
)

print("\nFormatted Prompt:\n", prompt)

# === 4. Tokenize and generate ===
inputs = tokenizer(prompt, return_tensors="pt", padding=True).to("xpu")

with torch.no_grad():
    output_ids = model.generate(
        inputs.input_ids,
        max_new_tokens=150,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

# === 5. Decode only the NEW part (after input) ===
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
response = generated_text[len(prompt):].strip()  # Remove input prompt

print("\nSummary:\n", response)

# === Cleanup ===
clearvram()

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Transcript:  Online accounts. Supposedly the night of the Trump call, we did a show in the evening. At that time, we used to record like at the end of the workday, like at five o'clock or so. The problem was John was already drunk. So we switched to 11 a.m. So we got hung over John instead of drunk John. So they would record at five o'clock and John would be shit face. Because if you remember back then, he was inviting and challenging people and saying, I'm at Pickwick Pub every day at three o'clock. Come come fight me. He wasn't lying. Yeah. But the night of the Trump call, we did an entire show. Somebody didn't show up. I don't know if it was. I really don't know. An ex stern guy. Grillo is the only one that comes to mind. And I apologize if I'm wrong. Sorry. Somebody else. But just in back and forth conversation, Royce says, you know, my cousin, Vinny thing, Jerry Callow, Jerry Gallo, John Melendez, Senator Menendez. So I'm sitting up at the board. So to the White House, we call. An

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.31s/it]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Formatted Prompt:
 <|system|>
You are a helpful assistant that summarizes podcast transcripts concisely.<|end|>
<|user|>
Please summarize the following podcast excerpt in 2-3 sentences:

 Online accounts. Supposedly the night of the Trump call, we did a show in the evening. At that time, we used to record like at the end of the workday, like at five o'clock or so. The problem was John was already drunk. So we switched to 11 a.m. So we got hung over John instead of drunk John. So they would record at five o'clock and John would be shit face. Because if you remember back then, he was inviting and challenging people and saying, I'm at Pickwick Pub every day at three o'clock. Come come fight me. He wasn't lying. Yeah. But the night of the Trump call, we did an entire show. Somebody didn't show up. I don't know if it was. I really don't know. An ex stern guy. Grillo is the only one that comes to mind. And I apologize if I'm wrong. Sorry. Somebody else. But just in back and forth conversati